### -------------Yanolja crawling---------------

In [52]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd

In [13]:
# 1. 프로젝트 생성

In [14]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [15]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
└── yanolja_hotel
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py

2 directories, 7 files


In [16]:
# 2. xpath 찾기 : headers 설정

In [54]:
import scrapy, requests #호텔 key값 얻는 모듈 생성 -> yanolja.py

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    checkin_date = "2021-03-05" #날짜 알고리즘
    checkout_date = "2021-03-06"
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(page))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [18]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

In [19]:
# 3. items.py : title, content, link, category

In [30]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [31]:
# 4. spider.py

In [47]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
            
    def parse_content(self, response):
        item = YanoljaHotelItem()
        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        item["location"] = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        item["name"] = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        item["level"] = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        item["score"] = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        item["review_count"] = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [48]:
# 5. settings.py : robots.txt 여부 파악 

In [49]:
# 6. 프로젝트 실행

In [50]:
%%writefile run.sh
cd yanolja_hotel
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [51]:
!/bin/bash run.sh

2021-03-05 18:59:11 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-05 18:59:11 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-05 18:59:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-05 18:59:11 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-05 18:59:11 [scrapy.extensions.telnet] INFO: Telnet Password: 7369ff500d187733
2021-03-05 18:59:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2021-03-05 18:59:14 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-05&checkoutDate=2021-03-06&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-05 18:59:14 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-05 18:59:15 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-05&checkoutDate=2021-03-06&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=13&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-05 18:59:15 [urllib3.connectionpool] DEBUG: Sta

2021-03-05 18:59:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3007477> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/1000101862> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001791> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001680> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3007477>
{'level': '2성급',
 'link': 'https://www.yanolja.com/hotel/3007477',
 'location': '서울특별시 영등포구 영등포로 389',
 'name': '베니키아 프리미어 여의도',
 'review_count': '1,969',
 'score': '4.6'}
2021-03-05 18:59:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000101862>
{'level': '4성급',
 

2021-03-05 18:59:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000112780>
{'level': '레지던스',
 'link': 'https://www.yanolja.com/hotel/1000112780',
 'location': '서울특별시 중구 충무로 24',
 'name': '어반스테이 더  남산',
 'review_count': '1',
 'score': '5.0'}
2021-03-05 18:59:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/1000110056> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:17 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.yanolja.com/hotel/1000110872> (referer: https://www.yanolja.com/hotel/)
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 120, in iter_errback
    yield next(it)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python

2021-03-05 18:59:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000109463>
{'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/1000109463',
 'location': '서울특별시 종로구 수표로22길 38',
 'name': '종로 엠앤럭키호텔',
 'review_count': '3',
 'score': '3.7'}
2021-03-05 18:59:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000100865>
{'level': '1성급',
 'link': 'https://www.yanolja.com/hotel/1000100865',
 'location': '서울 송파구 올림픽로12길 5-11',
 'name': '잠실 라몬호텔',
 'review_count': '4',
 'score': '4.7'}
2021-03-05 18:59:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3000840> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000963>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3000963',
 'location': '서울특별시 중구 퇴계로4길 69',
 'name': '시티호텔 서울',
 'review_count': '22',
 'score': '1.5'}
2021-03-05 18:59:18 [scrapy.core.e

...

2021-03-05 18:59:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3000698> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001154>
{'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/3001154',
 'location': '서울특별시 강남구 논현로87길 17',
 'name': '역삼 스타호텔',
 'review_count': '753',
 'score': '4.3'}
2021-03-05 18:59:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001030>
{'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3001030',
 'location': '서울특별시 중구 퇴계로20길 2',
 'name': '명동 퍼시픽 호텔',
 'review_count': '630',
 'score': '4.6'}
2021-03-05 18:59:34 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.yanolja.com/hotel/3014816> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3014794>
{'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/

2021-03-05 18:59:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3017023>
{'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3017023',
 'location': '서울특별시 서초구 청계산로9길 1-7',
 'name': '오라카이 청계산 호텔',
 'review_count': '760',
 'score': '4.6'}
2021-03-05 18:59:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001533> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001668> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001358> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001533>
{'level': '1성급',
 'link': 'https://www.yanolja.com/hotel/3001533',
 'location': '서울특별시 송파구 백제고분로 268',
 'name': '호텔 인피니',
 'review_count': '857',
 'score': '4.4'}
2021-03-05 1

2021-03-05 18:59:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001877> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3002236>
{'level': '2성급',
 'link': 'https://www.yanolja.com/hotel/3002236',
 'location': '서울특별시 성동구 천호대로 302',
 'name': '답십리역 호텔 이노스테이',
 'review_count': '960',
 'score': '4.6'}
2021-03-05 18:59:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3000648> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009264>
{'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009264',
 'location': '서울특별시 강서구 공항대로 30',
 'name': '로얄스퀘어 호텔 서울',
 'review_count': '881',
 'score': '4.7'}
2021-03-05 18:59:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000101565>
{'level': '4성급',
 'link': 'https://www.yanolja.c

2021-03-05 18:59:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009564>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3009564',
 'location': '서울특별시 동대문구 천호대로 383',
 'name': '더 리센츠 동대문 호텔',
 'review_count': '1,395',
 'score': '4.6'}
2021-03-05 18:59:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016266>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3016266',
 'location': '서울특별시 강남구 도산대로 216',
 'name': '에이든 바이 베스트웨스턴 청담',
 'review_count': '1,286',
 'score': '4.7'}
2021-03-05 18:59:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001558> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001558',
 'location': '서울특별시 마포구 마포대로 109',
 'name': '롯데시티호텔 마포',
 'review_count': '1,295',
 'score': '4.8'}
2021-03-05 18:59:37 [s

2021-03-05 18:59:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001397> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009050>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3009050',
 'location': '서울특별시 서초구 효령로 427',
 'name': '신라스테이 서초',
 'review_count': '2,184',
 'score': '4.6'}
2021-03-05 18:59:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001348> (referer: https://www.yanolja.com/hotel/)
2021-03-05 18:59:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001397>
{'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001397',
 'location': '경기도 화성시 노작로 161',
 'name': '신라스테이 동탄',
 'review_count': '1,614',
 'score': '4.7'}
2021-03-05 18:59:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001348>
{'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/

In [53]:
pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

,level,link,location,name,review_count,score
0,4성급,https://www.yanolja.com/hotel/3000586,서울특별시 마포구 양화로 112,메리골드 호텔,"2,267",4.4
1,3성급,https://www.yanolja.com/hotel/1000104461,서울특별시 중구 을지로 224,나인트리 호텔 동대문,"1,503",4.8
2,4성급,https://www.yanolja.com/hotel/3001075,서울특별시 영등포구 의사당대로 18,글래드 여의도,"1,767",4.7
3,3성급,https://www.yanolja.com/hotel/3000605,서울특별시 중구 동호로 354,라마다 서울 동대문,"1,318",4.5
4,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"8,716",4.7
5,3성급,https://www.yanolja.com/hotel/3001395,서울특별시 강남구 언주로 517,신라스테이 역삼,"1,793",4.7
6,4성급,https://www.yanolja.com/hotel/3000643,서울특별시 중구 마른내로 71,호텔 피제이 명동,"1,663",4.7
7,2성급,https://www.yanolja.com/hotel/3007477,서울특별시 영등포구 영등포로 389,베니키아 프리미어 여의도,"1,969",4.6
8,4성급,https://www.yanolja.com/hotel/1000101862,서울특별시 종로구 율곡로 180,오라카이 대학로 호텔,"1,691",4.8
9,비즈니스,https://www.yanolja.com/hotel/3001791,서울특별시 중구 장충단로13길 20,호텔 스카이파크 킹스타운 동대문점,"1,617",4.7


In [ ]:
# 7. pipeline.py : 크롤링한 데이터를 mysql에 저장